# Getting Started with APIs in Python

Note that the inspiration for this notebook comes from [this medium post.](https://towardsdatascience.com/getting-started-with-apis-in-python-to-gather-data-1185796b1ec3)
And here's the link to the [requests documentation.](https://requests.readthedocs.io/en/master/user/authentication/)

APIs, or Application Programming Interfaces, provide easy ways to retrieve (and post) data. They are the interfaces provided by servers that you can use to, among others, retrieve and send data using code.

They can be compared to a waiter at a restaurant. As a patron, you give the waiter instructions for a meal, who then communicates the request back to the kitchen, where the chefs undertake complex steps to prepare a dish, without you ever knowing all the hard work he or she put into it! You then get back the meal you ordered, without having to figure out the steps it took to get there!

##  Pros of APIs
> <li> More efficient than static downloads (automating data downloads)
> <li> Gives the ability to work with rapidly changing data
> <li> Adaptibility (can connect to APIs from most programming languages)
    
## How APIs work
When you want to work with an API, you need to make a request to the server (this is where the python requests library becomes useful!). A reqest is made up of 4 different points: <br>
> 1)  An **endpoint** - which looks like the URL to the data <br>
    2) A **method** - GET, PUT, POST, DELETE <br>
    3) The **headers** - which provide information such as authentication keys <br>
    4) the **data/body** - which is not applicable for GET methods<br>
    
Once we make the request, it will return a request object, which includes the data we're hoping to extract & also a status code. The status code will let us know if we connected successfully, or if there was some sort of error. Status codes are generally split up like this: 
> <li> 1xx: Provide information
> <li> 2xx: Generally are successful
> <li> 3xx: Provide information on redirects
> <li> 4xx: Refer to a client error (our bad)
> <li> 5xx: Refer to a server error (their bad)

# No Auth API

There's a great list of [public apis available on Github.](https://github.com/public-apis/public-apis) That's where I found the Punk API, which lists Brewdog Beer Recipies.


## Step 1: [Read the documentation](https://punkapi.com/documentation/v2) 
Never connect to an API without first reading the documentation. There are often rate limits, and if you don't follow the guidelines you can be blocked from the API pretty quickly. 

As you'll see, one of the first sections in the documentation is about Rate Limits. It then goes into detail on how you can access the data, which is perfect.

We can see that the root endpoint is https://api.punkapi.com/v2/, which means all calls will begin with this url, then we can add more parameters to get the data we want. 

In [9]:
import requests
import pandas as pd
import json
import time

In [252]:
# Define the URL Path
url = 'https://api.punkapi.com/v2/beers'

# Create the request object using the GET function
response = requests.get(url)

# Print the Response
print(response)

<Response [200]>


As per the documentation: 
> All parameters are optional and without them the API will just return the beers in ascending order from their ID.

So, we should receive the first 25 beers with the below code

In [253]:
# Print the JSON
beer_json = json.loads(response.text)

beer_json

[{'id': 1,
  'name': 'Buzz',
  'tagline': 'A Real Bitter Experience.',
  'first_brewed': '09/2007',
  'description': 'A light, crisp and bitter IPA brewed with English and American hops. A small batch brewed only once.',
  'image_url': 'https://images.punkapi.com/v2/keg.png',
  'abv': 4.5,
  'ibu': 60,
  'target_fg': 1010,
  'target_og': 1044,
  'ebc': 20,
  'srm': 10,
  'ph': 4.4,
  'attenuation_level': 75,
  'volume': {'value': 20, 'unit': 'litres'},
  'boil_volume': {'value': 25, 'unit': 'litres'},
  'method': {'mash_temp': [{'temp': {'value': 64, 'unit': 'celsius'},
     'duration': 75}],
   'fermentation': {'temp': {'value': 19, 'unit': 'celsius'}},
   'twist': None},
  'ingredients': {'malt': [{'name': 'Maris Otter Extra Pale',
     'amount': {'value': 3.3, 'unit': 'kilograms'}},
    {'name': 'Caramalt', 'amount': {'value': 0.2, 'unit': 'kilograms'}},
    {'name': 'Munich', 'amount': {'value': 0.4, 'unit': 'kilograms'}}],
   'hops': [{'name': 'Fuggles',
     'amount': {'value': 2

In [254]:
# Use the json normalize function to convert JSON to Pandas DF
beer_df = pd.json_normalize(beer_json)

# Ensures all columns show
pd.set_option('display.max_columns', None)

# View the Dataframe
beer_df.head(1)

,id,name,tagline,first_brewed,description,image_url,abv,ibu,target_fg,target_og,ebc,srm,ph,attenuation_level,food_pairing,brewers_tips,contributed_by,volume.value,volume.unit,boil_volume.value,boil_volume.unit,method.mash_temp,method.fermentation.temp.value,method.fermentation.temp.unit,method.twist,ingredients.malt,ingredients.hops,ingredients.yeast
0,1,Buzz,A Real Bitter Experience.,09/2007,"A light, crisp and bitter IPA brewed with Engl...",https://images.punkapi.com/v2/keg.png,4.5,60.0,1010,1044.0,20.0,10.0,4.4,75.0,"[Spicy chicken tikka masala, Grilled chicken q...",The earthy and floral aromas from the hops can...,Sam Mason <samjbmason>,20,litres,25,litres,"[{'temp': {'value': 64, 'unit': 'celsius'}, 'd...",19,celsius,None,"[{'name': 'Maris Otter Extra Pale', 'amount': ...","[{'name': 'Fuggles', 'amount': {'value': 25, '...",Wyeast 1056 - American Ale™


## Unnesting Pandas columns still in JSON format

You can see there are still some columns in list format because of how the JSON was nested. There are a few ways we can fix this, but here's one that has worked for me in the past. The only thing, is you need to do this manually for each column. I'm sure there's a more efficient way, but we will go with this for now. 

[Here's a great resource](https://towardsdatascience.com/all-pandas-json-normalize-you-should-know-for-flattening-json-13eae1dfb7dd)

In [257]:
# beer_df['ingredients.malt'][4]
beer_df['ingredients.malt'][10]

[{'name': 'Pale Ale', 'amount': {'value': 2.18, 'unit': 'kilograms'}},
 {'name': 'Caramalt', 'amount': {'value': 0.3, 'unit': 'kilograms'}},
 {'name': 'Dark Crystal', 'amount': {'value': 0.3, 'unit': 'kilograms'}},
 {'name': 'Smoked Weyermann', 'amount': {'value': 1.8, 'unit': 'kilograms'}},
 {'name': 'Flaked Oats', 'amount': {'value': 0.6, 'unit': 'kilograms'}},
 {'name': 'Brown', 'amount': {'value': 0.6, 'unit': 'kilograms'}},
 {'name': 'Amber', 'amount': {'value': 0.1, 'unit': 'kilograms'}},
 {'name': 'Chocolate', 'amount': {'value': 0.05, 'unit': 'kilograms'}},
 {'name': 'Munich', 'amount': {'value': 0.6, 'unit': 'kilograms'}},
 {'name': 'Crystal 150', 'amount': {'value': 0.2, 'unit': 'kilograms'}}]

# Passing Parameters

In [258]:
# Note that we defined our URL above, so we just need to pass the parameters
# Create the request object using the GET function
response = requests.get(f'{url}?food=pizza')

# Convert to JSON
pizza_json = json.loads(response.text)

# Convert to Dataframe
pizza_df = pd.json_normalize(pizza_json)

# View the Pizza df
pizza_df.head()

,id,name,tagline,first_brewed,description,image_url,abv,ibu,target_fg,target_og,ebc,srm,ph,attenuation_level,food_pairing,brewers_tips,contributed_by,volume.value,volume.unit,boil_volume.value,boil_volume.unit,method.mash_temp,method.fermentation.temp.value,method.fermentation.temp.unit,method.twist,ingredients.malt,ingredients.hops,ingredients.yeast
0,13,Movember,Moustache-Worthy Beer.,11/2009,"A deliciously robust, black malted beer with a...",https://images.punkapi.com/v2/13.png,4.5,50,1012,1047,140,70.00,5.2,74.5,"[Vegetable egg scramble, Margherita pizza, Fre...","If you can’t find really fresh cascade, substi...",Sam Mason <samjbmason>,20,litres,25,litres,"[{'temp': {'value': 68, 'unit': 'celsius'}, 'd...",19,celsius,None,"[{'name': 'Maris Otter Extra Pale', 'amount': ...","[{'name': 'Cascade', 'amount': {'value': 43.8,...",Wyeast 1056 - American Ale™
1,83,Comet,Single Hop India Pale Ale,02/2014,A potently bitter hop variety originally grown...,https://images.punkapi.com/v2/83.png,7.2,70,1012,1067,30,15.00,4.4,82.1,"[Margherita pizza with chili flakes, Spicy Tha...",Experiment with other high alpha hops during d...,Ali Skinner <AliSkinner>,20,litres,25,litres,"[{'temp': {'value': 65, 'unit': 'celsius'}, 'd...",19,celsius,None,"[{'name': 'Extra Pale', 'amount': {'value': 5....","[{'name': 'Comet', 'amount': {'value': 15, 'un...",Wyeast 1056 - American Ale™
2,178,Simcoe,Single Hop India Pale Ale.,01/2012,A special release of our IPA is Dead series - ...,https://images.punkapi.com/v2/178.png,6.7,70,1012,1063,30,15.00,4.4,81.0,"[Beer roasted chicken, Ham and pineapple pizza...",Get the freshest Simcoe for the best profile.,Sam Mason <samjbmason>,20,litres,25,litres,"[{'temp': {'value': 65, 'unit': 'celsius'}, 'd...",99,celsius,None,"[{'name': 'Extra Pale', 'amount': {'value': 4....","[{'name': 'Simcoe', 'amount': {'value': 2.5, '...",Wyeast 1056 - American Ale™
3,218,Monk Hammer,Our Ruthless IPA With A Belgian Twist.,03/2016,Jack Hammer has been single handedly ripping i...,https://images.punkapi.com/v2/218.png,7.2,250,1010,1065,15,7.50,4.4,84.6,"[Pesto chicken pizza, Beer braised Brussels sp...",Oxygen is critical for this strain of yeast so...,Sam Mason <samjbmason>,20,litres,25,litres,"[{'temp': {'value': 65, 'unit': 'celsius'}, 'd...",21,celsius,None,"[{'name': 'Extra Pale', 'amount': {'value': 5....","[{'name': 'Centennial', 'amount': {'value': 25...",Wyeast 3522 - Belgian Ardennes™
4,251,Small Batch: Sorachi Ace Session,Sorachi Ace Belgian Pale.,2016,This brew is a pale ale fermented with Belgian...,https://images.punkapi.com/v2/keg.png,4.0,25,1005,1035,15,7.62,5.1,79.0,"[Californian Sushi Roll, Parmesan and Rocket P...",If you dry hop with extra Sorachi Ace you'll e...,John Jenkman <johnjenkman>,20,litres,25,litres,"[{'temp': {'value': 65, 'unit': 'celsius'}, 'd...",19,celsius,Crushed Coridaner Seeds – FV,"[{'name': 'Pale Ale', 'amount': {'value': 2, '...","[{'name': 'Sorachi Ace', 'amount': {'value': 2...",Wyeast 1388 - Belgian Strong Ale™


# Pagination
As per the documentation, multiple items will be limited by 25 results. However, we can increase the amount of beers with the per_page parameter. Let's just loop through twice and get 100 beers total. 

In [268]:
# Create the empty dataframe
beers_df = pd.DataFrame()

# Define the first page
page_number = 1

# Initialize the while loop, only allowing two loops
while page_number < 3:

    # Create the request object using the GET function
    response = requests.get(f'{url}?page={page_number}&per_page=50')

    # Convert to JSON
    temp_json = json.loads(response.text)

    # Convert to Dataframe
    temp_df = pd.json_normalize(temp_json)

    # Append to the empty dataframe
    beers_df = beers_df.append(temp_df)
    
    # Update the page number by 1
    page_number += 1
    
    # Halt the code for 5 seconds to respect the rate limits
    time.sleep(5)

200
200


In [260]:
len(beers_df)

100

# Example Connections

### Basic Authentication

In [ ]:
# Auth credentials
auth = ('username', 'password')

# URL
url = 'www.someurl.com'

# Initial call to get total issues number
response = requests.get(url, auth = auth)

### OAuth 2

In [ ]:
from oauthlib.oauth2 import LegacyApplicationClient
from requests_oauthlib import OAuth2Session

# Apply Login info
oauth = OAuth2Session(client = LegacyApplicationClient(client_id = client_id))
token = oauth.fetch_token(token_url = token_url,
                          username = username, 
                          password = password,
                          client_id = client_id,
                          client_secret = client_secret,
                          scope = 'public,token.refresh')

# One Final Example
https://github.com/detectlanguage/detectlanguage-python

In [6]:
# Import detectlanguage package
import detectlanguage

# Define the API key
detectlanguage.configuration.api_key = api_key

# Detect the language of this sentence
detectlanguage.detect("Buenos dias señor")

[{'language': 'es', 'isReliable': True, 'confidence': 10.24}]

In [10]:
# Create list of names and sentences
data = [['Tom', 'The paintbrush was angry at the color the artist chose to use.'], 
        ['Nick', 'Haz lo que yo digo y no lo que yo hago'], 
        ['Amanda', 'Es freut mich, dich kennenzulernen']] 

# Create the dataframe from the list
df = pd.DataFrame(data, columns = ['Name', 'Sentence']) 

df

,Name,Sentence
0,Tom,The paintbrush was angry at the color the arti...
1,Nick,Haz lo que yo digo y no lo que yo hago
2,Amanda,"Es freut mich, dich kennenzulernen"


In [11]:
# Create an empty list to append to 
detection_list = []

# Loop through each sentence in the Sentence column
for sentence in df['Sentence']: 
    
    # Apply the detection method, append result to a list
    detection_list.append(detectlanguage.detect(sentence))

In [12]:
# Convert the list to a dataframe
detection_df = pd.DataFrame(detection_list)

# Use the .apply method to un-nest the dictionary
detection_df = detection_df[0].apply(pd.Series)

# Join the new dataframe back with the existing dataframe on the index
df_merged = df.merge(detection_df, how='inner', left_index=True, right_index=True)

df_merged

,Name,Sentence,language,isReliable,confidence
0,Tom,The paintbrush was angry at the color the arti...,en,True,13.54
1,Nick,Haz lo que yo digo y no lo que yo hago,es,True,9.71
2,Amanda,"Es freut mich, dich kennenzulernen",de,True,9.33
